# Detector Models Comparison

This notebook demonstrates how to compare different detector models in G4LumaCam using the `groupby` functionality.

In [ ]:
from lumacam import Config, Simulate, Lens, Analysis
import matplotlib.pyplot as plt
import pandas as pd
from pathlib import Path

## 1. Run Simulation

In [ ]:
archive = "demo_models"

config = Config.neutrons_tof(energy_min=1.0, energy_max=10.0)
config.num_events = 1000

sim = Simulate(archive=archive)
sim.run(config)

## 2. Group by Detector Models

Define multiple detector model configurations and group by them.

In [ ]:
lens = Lens(archive=archive)

lens.groupby("detector_model", bins=[
    {"name": "intensifier", "detector_model": "image_intensifier", "blob": 2.0, "deadtime": 600},
    {"name": "intensifier_gain", "detector_model": "image_intensifier_gain", "gain": 5000, "blob": 0, "deadtime": 475},
    {"name": "full_physics", "detector_model": "physical_mcp", "gain": 8000, "phosphor_type": "p47", "blob": 0, "deadtime": 475}
])

## 3. Trace Rays

Automatically traces rays for all detector models.

In [ ]:
lens.trace_rays(seed=42)

## 4. Process with EMPIR

Automatically detects grouped structure and processes all models.

In [ ]:
analysis = Analysis(archive=f"{archive}/detector_model")

analysis.process(
    params="hitmap",
    event2image=True,
    export_photons=True,
    export_events=True,
    export_pixels=True
)

## 5. Compare Results

Load and compare exported data from each model.

In [ ]:
# Load photon data from each model
model_names = ["intensifier", "intensifier_gain", "full_physics"]
photon_data = {}

for name in model_names:
    photon_dir = Path(archive) / "detector_model" / name / "ExportedPhotons"
    if photon_dir.exists():
        files = list(photon_dir.glob("*.csv"))
        if files:
            photon_data[name] = pd.read_csv(files[0])

# Compare photon statistics
for name, df in photon_data.items():
    print(f"\n{name}:")
    print(f"  Total photons: {len(df)}")
    if 'tot' in df.columns:
        print(f"  Mean TOT: {df['tot'].mean():.2f} ns")
        print(f"  Std TOT: {df['tot'].std():.2f} ns")

## 6. Visualize TOT Distributions

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

for ax, (name, df) in zip(axes, photon_data.items()):
    if 'tot' in df.columns:
        ax.hist(df['tot'], bins=50, alpha=0.7, edgecolor='black')
        ax.set_xlabel('TOT (ns)')
        ax.set_ylabel('Count')
        ax.set_title(name)
        ax.grid(alpha=0.3)

plt.tight_layout()
plt.show()

## 7. Single Event Visualization

In [ ]:
try:
    import visualize_pixel_map as vpm
    import numpy as np
    
    # Load event data from one model
    event_dir = Path(archive) / "detector_model" / "intensifier_gain" / "ExportedEvents"
    if event_dir.exists():
        files = list(event_dir.glob("*.csv"))
        if files:
            df = pd.read_csv(files[0])
            
            # Plot first 5 events
            if 'event_id' in df.columns and len(df) > 0:
                event_ids = df['event_id'].unique()[:5]
                fig, axes = plt.subplots(1, 5, figsize=(20, 4))
                
                for ax, event_id in zip(axes, event_ids):
                    event_df = df[df['event_id'] == event_id]
                    pixel_map = np.zeros((256, 256))
                    for _, row in event_df.iterrows():
                        x, y = int(row['pixel_x']), int(row['pixel_y'])
                        if 0 <= x < 256 and 0 <= y < 256:
                            pixel_map[y, x] += 1
                    vpm.plot_pixel_map(pixel_map, ax=ax, title=f"Event {event_id}")
                
                plt.tight_layout()
                plt.show()
except ImportError:
    print("visualize_pixel_map not installed")
    print("Install with: pip install git+https://github.com/TsvikiHirsh/visualize_pixel_map.git")

## Folder Structure

```
demo_models/
├── SimPhotons/
└── detector_model/
    ├── intensifier/
    │   ├── tpx3Files/
    │   ├── ExportedPhotons/
    │   ├── ExportedEvents/
    │   └── ExportedPixels/
    ├── intensifier_gain/
    │   └── ...
    └── full_physics/
        └── ...
```